In [1]:
#once I was able to clean my initial data set I found I had access to ingredient data about the beers in
#the API so I recreated the data set with more informaiton.
import requests
import re
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: No module named seaborn

In [113]:
rows = []

def parse_beer(beer):
    breweries = beer.get('breweries', [{}])
    brewery = breweries[0]
    locations = brewery.get('locations', [{}])
    location = locations[0]
    beers = [beer.get('id'),
             beer.get('name'),
             beer.get('abv'),
             beer.get('ibu'),
             beer.get('originalGravity'),
             beer.get('style',{}).get('ogMin'),
             beer.get('isOrganic'),
             beer.get('style', {}).get('categoryId'),
             beer.get('style', {}).get('category', {}).get('name'),
             beer.get('styleId'),
             beer.get('style', {}).get('name'),
             beer.get('style', {}).get('description'),
             beer.get('servingTemperatureDisplay'),
             beer.get('foodPairings'),
             beer.get('description'),
             beer.get('labels', {}).get('medium'),
             beer.get('glasswareId'),
             beer.get('glass', {}).get('name'),
             beer.get('srmId'),
             beer.get('srm', {}).get('hex'),
             beer.get('style', {}).get('srmMin'),
             beer.get('style', {}).get('srmMax'),            
             brewery.get('id'),
             brewery.get('name'),
             brewery.get('description'),
             brewery.get('established'),
             location.get('locality'),
             location.get('country', {}).get('numberCode')]

    return beers
        


pages = range(1,1182)
for page in pages:
    r = requests.get('http://api.brewerydb.com/v2/beers/?key=209db383ed1d39ec3aa1c13f89161c25&p='+str(page)+'&withBreweries=Y&withIngredients=Y')
    response = r.json()
    rows += [parse_beer(r) for r in response['data']]
    
 

In [114]:
beer = pd.DataFrame(rows)

In [115]:
#59,000 beers! 
beer.shape

(59050, 28)

In [116]:
beer.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,cBLTUw,"""18"" Imperial IPA 2",11.1,None,None,1.08,N,3.0,North American Origin Ales,43.0,American-Style Imperial Stout,"Black in color, American-style imperial stouts...",None,None,Hop Heads this one's for you! Checking in wit...,None,5.0,Pint,33.0,520907,40,40,qa1QZU,Ship Bottom Brewery,Our humble beginnings started in the Summer of...,2011,Wallingford,840.0
1,ZsQEJt,"""633"" American Pale Ale",6.33,25,None,1.044,N,3.0,North American Origin Ales,25.0,American-Style Pale Ale,American pale ales range from deep golden to c...,None,None,"Our first beer has been aptly named ""633"" afte...",None,NaN,None,NaN,None,6,14,DBkLHq,"Boothbay Craft Brewery, Inc",The 1st brewery in the Boothbay Region. Our id...,None,Boothbay,840.0
2,tmEthz,"""Admiral"" Stache",7,23,1.071,1.072,N,9.0,Other Lager,104.0,Baltic-Style Porter,A true smooth cold-fermented and cold lagered ...,None,None,Milwaukee Brewing Co’s take on a classic Europ...,https://s3.amazonaws.com/brewerydbapi/beer/tmE...,5.0,Pint,37.0,3F0708,40,40,BU4IJP,Milwaukee Brewing Company,The Milwaukee Brewing Company is the 'grown up...,1997,Milwaukee,840.0


In [117]:
#sadly there was still a lot of missing information.
beer.isnull().sum()

0         0
1         0
2     11010
3     38270
4     54366
5      7278
6         1
7      2684
8      2684
9      2684
10     2684
11     2868
12    55576
13    55945
14    23632
15    37299
16    43274
17    43274
18    53147
19    53147
20     8215
21     8260
22       29
23       29
24    10961
25    13687
26     1752
27       81
dtype: int64

In [118]:
beer.columns = ['id',
                'name',
                'abv',
                'ibu',
                'originalGravity',
                'ogMin',
                'isOrganic',
                'categoryId',
                'category_name',
                'styleId',
                'style_name',
                'style_description',
                'servingTemperatureDisplay',
                'foodPairings',
                'description',
                'labels',
                'glasswareId',
                'glass_name',
                'srmId',
                'srm_hex',
                'srmMin',
                'srmMax',
                'brewery_id',
                'brewery_name',
                'brewery_description',
                'brewery_established',
                'brewery_locality',
                'loc_country_code']

In [119]:
beer.to_csv('beers_full.csv', encoding = 'utf-8')

In [2]:
beer = pd.read_csv('beers_full.csv')

In [4]:
beer.head(3)

,Unnamed: 0,id,name,abv,ibu,originalGravity,ogMin,isOrganic,categoryId,category_name,...,srmId,srm_hex,srmMin,srmMax,brewery_id,brewery_name,brewery_description,brewery_established,brewery_locality,loc_country_code
0,0,cBLTUw,"""18"" Imperial IPA 2",11.10,NaN,NaN,1.080,N,3.0,North American Origin Ales,...,33.0,520907,40.0,40.0,qa1QZU,Ship Bottom Brewery,Our humble beginnings started in the Summer of...,2011.0,Wallingford,840.0
1,1,ZsQEJt,"""633"" American Pale Ale",6.33,25.0,NaN,1.044,N,3.0,North American Origin Ales,...,NaN,NaN,6.0,14.0,DBkLHq,"Boothbay Craft Brewery, Inc",The 1st brewery in the Boothbay Region. Our id...,NaN,Boothbay,840.0
2,2,tmEthz,"""Admiral"" Stache",7.00,23.0,1.071,1.072,N,9.0,Other Lager,...,37.0,3F0708,40.0,40.0,BU4IJP,Milwaukee Brewing Company,The Milwaukee Brewing Company is the 'grown up...,1997.0,Milwaukee,840.0


In [3]:
#now get hop, malt, and malt characteristic inforamtion from the API for the beers that have it available.
for n in range(1,1183):
    print n
    r = requests.get('http://api.brewerydb.com/v2/beers/?key=209db383ed1d39ec3aa1c13f89161c25&p='+str(n)+'&withBreweries=Y&withIngredients=Y')
    data = r.json().get('data')
    for i in range(0,len(data)):
        d = data[i]
        ingredients = d.get('ingredients', {})
        
        if len(ingredients) > 0:
            hops = ingredients.get('hops', [{}])
            for n in range(0,len(hops)):
                hop = hops[n]
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_name'] = hop.get('name')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_id'] = hop.get('id')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_alphaAcidMin'] = hop.get('alphaAcidMin')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_betaAcidMax'] = hop.get('betaAcidMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_betaAcidMin'] = hop.get('betaAcidMin')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_caryophylleneMax'] = hop.get('caryophylleneMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_caryophylleneMin'] = hop.get('caryophylleneMin')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_cohumuloneMax'] = hop.get('cohumuloneMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_cohumuloneMin'] = hop.get('cohumuloneMin')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_description'] = hop.get('description')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_farneseneMax'] = hop.get('farneseneMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_humuleneMax'] = hop.get('humuleneMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_humuleneMin'] = hop.get('humuleneMin')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_myrceneMax'] = hop.get('myrceneMax')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'hop'+str(n)+'_myrceneMin'] = hop.get('myrceneMin')
            malts = ingredients.get('malt', [{}])
            for m in range(0,len(malts)):
                malt = malts[m]
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_name'] = malt.get('name')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_id'] = malt.get('id')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_description'] = malt.get('description')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_coarseFineDifference'] = malt.get('coarseFineDifference')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_country_code'] = malt.get('country_code')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_dryYield'] = malt.get('dryYield')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_moistureContent'] = malt.get('moistureContent')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_maxInBatch'] = malt.get('maxInBatch')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_potential'] = malt.get('potential')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_protien'] = malt.get('protien')
                beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_srmPrecise'] = malt.get('srmPrecise')
            for m in range(0,len(malts)):
                malt = malts[m]
                chars = malt.get('characteristics', [{}])
                for l in range(0,len(chars)):
                    char = chars[l]
                    beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_char'+str(l)+'_name'] = char.get('name')
                    beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_char'+str(l)+'_id'] = char.get('id')
                    beer.loc[beer[beer['id'] == str(d.get('id'))].index[0], 'malt'+str(m)+'_char'+str(l)+'_description'] = char.get('description')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [6]:
#check what we got.
beer.head(3)


,Unnamed: 0,id,name,abv,ibu,originalGravity,ogMin,isOrganic,categoryId,category_name,...,malt2_char0_description,malt2_char1_name,malt2_char1_id,malt2_char1_description,malt3_char0_name,malt3_char0_id,malt3_char0_description,malt0_char1_name,malt0_char1_id,malt0_char1_description
0,0,cBLTUw,"""18"" Imperial IPA 2",11.10,NaN,NaN,1.080,N,3.0,North American Origin Ales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ZsQEJt,"""633"" American Pale Ale",6.33,25.0,NaN,1.044,N,3.0,North American Origin Ales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,tmEthz,"""Admiral"" Stache",7.00,23.0,1.071,1.072,N,9.0,Other Lager,...,Imparts a malty flavor upon the beer.,Sweet,14.0,Imparts a sweet flavor upon the beer.,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
beer.to_csv('beers_full2.csv', encoding = 'utf-8')

In [179]:
#create a dictionary of all the hop information.
rows = {'hop_name': [],
        'hop_id': [],
        'hop_alphaAcidMin':[],
        'hop_betaAcidMax': [],
        'hop_betaAcidMin': [],
        'hop_caryophylleneMax': [],
        'hop_caryophylleneMin': [],
        'hop_cohumuloneMax': [],
        'hop_cohumuloneMin': [],
        'hop_description': [],
        'hop_farneseneMax': [],
        'hop_humuleneMax': [],
        'hop_humuleneMin': [],
        'hop_myrceneMax': [],
        'hop_myrceneMin': []}


for index, row in beer.iterrows():
    for n in range(0,13):
        if row['hop'+str(n)+'_name'] > 0:
            print row['hop'+str(n)+'_name']
            rows['hop_name'].append(row['hop'+str(n)+'_name'])
            rows['hop_id'].append(row['hop'+str(n)+'_id'])
            rows['hop_alphaAcidMin'].append(row['hop'+str(n)+'_alphaAcidMin'])
            rows['hop_betaAcidMax'].append(row['hop'+str(n)+'_betaAcidMax'])
            rows['hop_betaAcidMin'].append(row['hop'+str(n)+'_betaAcidMin'])
            rows['hop_caryophylleneMax'].append(row['hop'+str(n)+'_caryophylleneMax'])
            rows['hop_caryophylleneMin'].append(row['hop'+str(n)+'_caryophylleneMin'])
            rows['hop_cohumuloneMax'].append(row['hop'+str(n)+'_cohumuloneMax'])
            rows['hop_cohumuloneMin'].append(row['hop'+str(n)+'_cohumuloneMin'])
            rows['hop_description'].append(row['hop'+str(n)+'_description'])
            rows['hop_farneseneMax'].append(row['hop'+str(n)+'_farneseneMax'])
            rows['hop_humuleneMax'].append(row['hop'+str(n)+'_humuleneMax'])
            rows['hop_humuleneMin'].append(row['hop'+str(n)+'_humuleneMin'])
            rows['hop_myrceneMax'].append(row['hop'+str(n)+'_myrceneMax'])
            rows['hop_myrceneMin'].append(row['hop'+str(n)+'_myrceneMin'])
        



Perle (American)
Saaz (American)
Cascade
Sterling
Apollo
Cascade
Cascade
Sorachi Ace
Mount Hood
Apollo
Chinook
Horizon
Simcoe
Fuggle (American)
Fuggle (English)
East Kent Golding
Fuggle (English)
Northdown
Columbus
Glacier
Simcoe
Amarillo
Mosaic
Nugget
Fuggles
Northdown
Saaz (American)
Golding (American)
Columbus
Glacier
Horizon
Nugget
Simcoe
Glacier
Golding (American)
Cascade
Centennial
Chinook
Columbus
Bravo
Ahtanum
Amarillo
CTZ
East Kent Golding
Glacier
Target
Cascade
Centennial
Chinook
Golding (American)
Willamette
Amarillo
Golding (American)
Nelson Sauvin
Willamette
Amarillo
Centennial
Galena
Saaz (American)
Amarillo
Centennial
Sterling
Citra
Galaxy
Green Bullet
Mosaic
Nugget
Simcoe
Centennial
Citra
Columbus
Pacific Jade
Simcoe
German Polaris
East Kent Golding
Fuggles
Cascade
Columbus
Falconer's Flight
Cascade
Tettnang Tettnanger
Citra
Simcoe
Apollo
Willamette
Cascade
Northern Brewer (American)
Golding (American)
Aged / Debittered Hops (Lambic)
Liberty
Willamette
Centennial
Chinoo

In [180]:
hop_list = pd.DataFrame(rows)

In [326]:
hop_list = hop_list.drop_duplicates(subset = 'hop_id')

In [327]:
#now have a data set of all the hop information with no duplicates.
hop_list.to_csv('hop_list.csv', encoding = 'utf-8')

In [285]:
#do the same for malt information
rows2 = {'malt_name': [],
        'malt_id': [],
        'malt_description':[],
        'malt_coarseFineDifference': [],
        'malt_country_code': [],
        'malt_dryYield': [],
        'malt_moistureContent': [],
        'malt_maxInBatch': [],
        'malt_potential': [],
        'malt_protien': [],
        'malt_srmPrecise': [],
       }


for index, row in beer.iterrows():
    for n in range(0,10):
        if row['malt'+str(n)+'_name'] > 0:
            print row['malt'+str(n)+'_name']
            rows2['malt_name'].append(row['malt'+str(n)+'_name'])
            rows2['malt_id'].append(row['malt'+str(n)+'_id'])
            rows2['malt_description'].append(row['malt'+str(n)+'_description'])
            rows2['malt_coarseFineDifference'].append(row['malt'+str(n)+'_coarseFineDifference'])
            rows2['malt_country_code'].append(row['malt'+str(n)+'_country_code'])
            rows2['malt_dryYield'].append(row['malt'+str(n)+'_dryYield'])
            rows2['malt_moistureContent'].append(row['malt'+str(n)+'_moistureContent'])
            rows2['malt_maxInBatch'].append(row['malt'+str(n)+'_maxInBatch'])
            rows2['malt_potential'].append(row['malt'+str(n)+'_potential'])
            rows2['malt_protien'].append(row['malt'+str(n)+'_protien'])
            rows2['malt_srmPrecise'].append(row['malt'+str(n)+'_srmPrecise'])
            
            

Barley - Malted
Chocolate Malt
Munich Malt
Oats - Flaked
Honey Malt
Two-Row Pale Malt
Caramel/Crystal Malt
Pale Malt
Chocolate Malt
Munich Malt
Two-Row Pale Malt - Organic
Blackprinz Malt
Caramel/Crystal Malt
Two-Row Pale Malt - Organic
Caramel/Crystal Malt
Chocolate Malt
Munich Malt
Two-Row Pale Malt - Organic
Barley - Roasted
Caramel/Crystal Malt
Chocolate Malt
Two-Row Pale Malt - Organic
Barley - Roasted
Black Malt
Caramel/Crystal Malt
CaraMunich
Chocolate Malt
Maris Otter
Two-Row Pale Malt - Organic
Caramel/Crystal Malt
Chocolate Malt
Maris Otter
Wheat Malt
Caramel/Crystal Malt
Two-Row Pale Malt - Organic
Wheat Malt
Aromatic Malt
CaraMunich
Two-Row Pale Malt - Organic
Caramel/Crystal Malt
Two-Row Pale Malt - Organic
Wheat Malt
Caramel/Crystal Malt
Two-Row Pale Malt - Organic
Wheat Malt
CaraMunich II
Pale Malt - Organic
Special B Malt
Oats - Malted
Pilsner Malt
Wheat Malt
Caramel/Crystal Malt
Two-Row Pale Malt - Organic
Wheat Malt
Black Malt
Caramel/Crystal Malt
Chocolate Malt
Two-R

In [306]:
malt_list = pd.DataFrame(rows2)



,malt_coarseFineDifference,malt_country_code,malt_description,malt_dryYield,malt_id,malt_maxInBatch,malt_moistureContent,malt_name,malt_potential,malt_protien,malt_srmPrecise
0,NaN,NaN,None,NaN,1947.0,NaN,NaN,Barley - Malted,NaN,NaN,NaN
1,1.5,NaN,A dark malt that gives a rich red or brown col...,60.0,429.0,10.0,4.0,Chocolate Malt,1.028,NaN,350.0
2,1.3,NaN,Munich Malt has a malty sweet flavor character...,80.0,650.0,80.0,5.0,Munich Malt,1.037,NaN,9.0
3,NaN,NaN,None,NaN,536.0,NaN,NaN,Oats - Flaked,NaN,NaN,NaN
4,2.0,NaN,This Canadian malt imparts a honey-like flavor...,80.0,589.0,10.0,3.8,Honey Malt,1.037,NaN,25.0
5,1.5,NaN,The is the standard starting point for almost ...,79.0,1922.0,100.0,4.0,Two-Row Pale Malt,1.036,NaN,2.0
6,NaN,NaN,None,NaN,378.0,NaN,NaN,Caramel/Crystal Malt,NaN,NaN,NaN
7,NaN,NaN,None,NaN,234.0,NaN,NaN,Pale Malt,NaN,NaN,NaN
10,NaN,NaN,None,NaN,306.0,NaN,NaN,Two-Row Pale Malt - Organic,NaN,NaN,NaN
11,NaN,NaN,None,NaN,271.0,NaN,NaN,Blackprinz Malt,NaN,NaN,NaN


In [309]:
malt_list = malt_list.drop_duplicates(subset = 'malt_id')

In [328]:
malt_list.to_csv('malt_list.csv', encoding = 'utf-8')

In [323]:
#and the same with malt characteristics.
rows3 = {'malt_name': [],
         'malt_id': [],
         'malt_char_name': [],
         'malt_char_id': [],
         'malt_char_description': []
        }
    

for index, row in beer.iterrows():
    for n in range(0,10):
        if row['malt'+str(n)+'_char0_name'] > 0:
            rows3['malt_name'].append(row['malt'+str(n)+'_name'])
            rows3['malt_id'].append(row['malt'+str(n)+'_id'])
            rows3['malt_char_name'].append(row['malt'+str(n)+'_char0_name'])
            rows3['malt_char_id'].append(row['malt'+str(n)+'_char0_id'])
            rows3['malt_char_description'].append(row['malt'+str(n)+'_char0_description'])
        elif row['malt'+str(n)+'_char1_name'] > 0:
            rows3['malt_name'].append(row['malt'+str(n)+'_name'])
            rows3['malt_id'].append(row['malt'+str(n)+'_id'])
            rows3['malt_char_name'].append(row['malt'+str(n)+'_char1_name'])
            rows3['malt_char_id'].append(row['malt'+str(n)+'_char1_id'])
            rows3['malt_char_description'].append(row['malt'+str(n)+'_char1_description'])
        elif row['malt'+str(n)+'_char2_name'] > 0:
            rows3['malt_name'].append(row['malt'+str(n)+'_name'])
            rows3['malt_id'].append(row['malt'+str(n)+'_id'])
            rows3['malt_char_name'].append(row['malt'+str(n)+'_char2_name'])
            rows3['malt_char_id'].append(row['malt'+str(n)+'_char2_id'])
            rows3['malt_char_description'].append(row['malt'+str(n)+'_char2_description'])
        elif row['malt'+str(n)+'_char3_name'] > 0:
            rows3['malt_name'].append(row['malt'+str(n)+'_name'])
            rows3['malt_id'].append(row['malt'+str(n)+'_id'])
            rows3['malt_char_name'].append(row['malt'+str(n)+'_char3_name'])
            rows3['malt_char_id'].append(row['malt'+str(n)+'_char3_id'])
            rows3['malt_char_description'].append(row['malt'+str(n)+'_char3_description'])
    for m in range(0,7):
        if row['malt'+str(m)+'_char4_name'] > 0:
            rows3['malt_name'].append(row['malt'+str(m)+'_name'])
            rows3['malt_id'].append(row['malt'+str(m)+'_id'])
            rows3['malt_char_name'].append(row['malt'+str(m)+'_char4_name'])
            rows3['malt_char_id'].append(row['malt'+str(m)+'_char4_id'])
            rows3['malt_char_description'].append(row['malt'+str(m)+'_char4_description'])


In [324]:
char_list = pd.DataFrame(rows3)

In [329]:
char_list = char_list.drop_duplicates(subset = 'malt_id')

In [330]:
char_list.to_csv('char_list.csv', encoding = 'utf-8')